# **Indonesian Sentiment Analysis**
The project implements sentiment analysis for Indonesian-language text, classifying it into three categories: **negative**, **neutral**, and **positive**. The workflow is powered by TensorFlow Extended (TFX), ensuring reproducibility, scalability, and adaptability. The pipeline includes components for data ingestion, preprocessing, model training, evaluation, and deployment.

In [1]:
# Suppresses INFO, WARNING, and ERROR logs.
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')


In [2]:
# File handling and HTTP requests modules.
import requests

# Data manipulation and analysis tools.
import pandas as pd

# Machine learning and model evaluation frameworks.
import tensorflow as tf
import tensorflow_model_analysis as tfma

# TFX components for building and managing the machine learning pipeline.
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Trainer,
    Tuner,
    Evaluator,
    Pusher,
)

# TFX protocol buffers for configuring pipeline components.
from tfx.proto import (
    example_gen_pb2,
    trainer_pb2,
    pusher_pb2,
)

# Interactive execution of TFX components in a notebook environment.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Resolver and strategy for selecting the latest blessed model.
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy

# TFX types and standard artifacts for defining pipeline inputs and outputs.
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


## **1. Data Preparation**
- **Objective**: Download, preprocess, and organize datasets for training, evaluation, and testing.
- **Dataset**:
  - Source: [SmSA IndoNLU Dataset](https://github.com/IndoNLP/indonlu/tree/ce728f6926a36174b9923dfe49d6a6839b6e9bb7/dataset/smsa_doc-sentiment-prosa) [(Purwarianti and Crisdayanti, 2019)](https://arxiv.org/pdf/2009.05720)
  - Sentiment categories: `negative`, `neutral`, `positive`.
- **Preprocessing**:
  - Convert `.tsv` files into `.csv`.
  - Map sentiment labels to numerical values:
    - `negative` → `0`
    - `neutral` → `1`
    - `positive` → `2`.
- **Output**: Cleaned datasets stored in `train`, `eval`, and `test` directories.

In [3]:
# Create the 'data' folder.
DATA_ROOT = 'data'
os.makedirs(DATA_ROOT, exist_ok=True)

# List of file names and their corresponding subfolders.
files_and_folders = [
    {'file': 'train_preprocess.tsv', 'folder': 'train'},
    {'file': 'valid_preprocess.tsv', 'folder': 'eval'},
    {'file': 'test_preprocess.tsv', 'folder': 'test'}
]

# Base URL for raw TSV files.
base_url = 'https://github.com/IndoNLP/indonlu/raw/ce728f6926a36174b9923dfe49d6a6839b6e9bb7/dataset/smsa_doc-sentiment-prosa/'

# Define the manual mapping for sentiment labels.
sentiment_mapping = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

# Download and process each file.
for item in files_and_folders:
    file_name = item['file']
    folder_name = item['folder']

    # Create the subfolder.
    subfolder_path = os.path.join(DATA_ROOT, folder_name)
    os.makedirs(subfolder_path, exist_ok=True)

    # Construct the full URL for the file.
    file_url = base_url + file_name

    # Download the file.
    response = requests.get(file_url)
    if response.status_code == 200:
        # Save the file in the appropriate subfolder.
        tsv_path = os.path.join(subfolder_path, file_name)
        with open(tsv_path, 'wb') as f:
            f.write(response.content)

        # Read the TSV file into a DataFrame and add column names.
        df = pd.read_table(
            tsv_path, 
            sep='\t', 
            header=None, 
            names=['text', 'sentiment']
        )

        # Map sentiment labels to numerical values.
        df['sentiment'] = df['sentiment'].map(sentiment_mapping)

        # Save the DataFrame as CSV.
        csv_path = tsv_path.replace('.tsv', '.csv')
        df.to_csv(csv_path, index=False)
        print(f'Processed and saved CSV: {csv_path}')

        # Remove the TSV file.
        os.remove(tsv_path)
    else:
        print(f'Failed to download: {file_name}')

# Print the Label Mapping
print('Label Mapping:', sentiment_mapping)


Processed and saved CSV: data/train/train_preprocess.csv
Processed and saved CSV: data/eval/valid_preprocess.csv
Processed and saved CSV: data/test/test_preprocess.csv
Label Mapping: {'negative': 0, 'neutral': 1, 'positive': 2}


## **2. Pipeline Initialization**
- **Pipeline Components**:
  - `PIPELINE_NAME`: Specifies the pipeline name.
  - `PIPELINE_ROOT`: Directory to store artifacts generated during the pipeline execution.
  - `METADATA_PATH`: Database for tracking pipeline execution details.
- **Interactive Context**: Initializes a TFX environment for component execution and debugging.


In [4]:
# Name of the pipeline for sentiment analysis.
PIPELINE_NAME = 'sentiment_pipeline'
SCHEMA_PIPELINE_NAME = 'tfdv-schema'

# Root directory for storing pipeline artifacts and outputs.
PIPELINE_ROOT = os.path.join('pipeline', PIPELINE_NAME)

# Path to the metadata store for tracking pipeline executions and artifacts.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Directory for saving the trained model for serving.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)


In [5]:
# Initialize the interactive TFX context.
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## **3. Data Statistics and Schema Generation**
- **Components**:
  - `CsvExampleGen`: Ingests and splits the dataset into `train`, `eval`, and `test`.
  - `StatisticsGen`: Computes descriptive statistics for data analysis.
  - `SchemaGen`: Automatically generates a schema based on the statistics.
  - `ExampleValidator`: Detects anomalies or missing values.
- **Outputs**:
  - Data statistics visualization.
  - Inferred schema highlighting feature types and constraints.

In [6]:
# Configure the input splits for training and evaluation datasets.
input_config = example_gen_pb2.Input(splits=[
        example_gen_pb2.Input.Split(name='train', pattern='train/*'),
        example_gen_pb2.Input.Split(name='eval', pattern='eval/*'),
])

# Create the ExampleGen component for ingesting CSV files.
example_gen = CsvExampleGen(input_base=DATA_ROOT, input_config=input_config)

# Run the ExampleGen component to generate examples.
interactive_context.run(example_gen)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
# Create the StatisticsGen component to compute statistics for the dataset.
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

# Run the StatisticsGen component to calculate statistics.
interactive_context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
# Visualize the computed statistics from StatisticsGen.
interactive_context.show(statistics_gen.outputs['statistics'])

In [9]:
# Create the SchemaGen component to infer the schema from the statistics.
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])

# Run the SchemaGen component to generate the schema.
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
# Visualize the generated schema from SchemaGen.
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'sentiment',INT,required,,-
'text',BYTES,required,,-


In [11]:
# Create the ExampleValidator component to identify data anomalies. 
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

# Run the ExampleValidator component to detect anomalies.
interactive_context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
# Visualize the detected anomalies from ExampleValidator.
interactive_context.show(example_validator.outputs['anomalies'])

## **4. Data Transformation**
- **Objective**: Preprocess raw text into a format suitable for machine learning.
- **Transformations**:
  - Standardize text to lowercase.
  - Remove non-alphanumeric characters and stop words.
  - Strip unnecessary whitespace.
- **Component**:
  - `Transform`: Applies preprocessing logic using TFX.
- **Outputs**:
  - Transformed datasets for training and evaluation.
  - Transformation graph for consistent data handling during serving.

In [13]:
# Define the file name for the transform module.
TRANSFORM_MODULE_FILE = 'sentiment_transform.py'

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}

# Import required libraries.
import tensorflow as tf
import nltk
from nltk.corpus import stopwords

# Download stopwords and load them for preprocessing.
nltk.download('stopwords')
stopwords = list(stopwords.words('indonesian'))

# Define feature and label keys.
LABEL_KEY = 'sentiment'
FEATURE_KEY = 'text'

def transformed_name(key):
    '''Rename transformed features.'''
    return key + '_xf'

def preprocessing_fn(inputs):
    '''
    Preprocess input features into transformed features.

    Args:
        inputs: Map from feature keys to raw features.

    Returns:
        outputs: Map from feature keys to transformed features.
    '''
    outputs = {}

    # Standardize and clean text data.
    text = tf.strings.lower(inputs[FEATURE_KEY])
    text = tf.strings.regex_replace(text, r'[^a-z\s]', ' ')
    text = tf.strings.regex_replace(text, r'\b(' + r'|'.join(stopwords) + r')\b\s*', ' ')
    text = tf.strings.strip(text)
    
    # Assign transformed features to outputs.
    outputs[transformed_name(FEATURE_KEY)] = text
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs


Writing sentiment_transform.py


In [15]:
# Initialize the Transform component with the specified module file and inputs.
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

# Run the Transform component to preprocess and transform the data.
interactive_context.run(transform)


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying sentiment_transform.py -> build/lib
installing to /tmp/tmp0d53p2a4
running install
running install_lib
copying build/lib/sentiment_transform.py -> /tmp/tmp0d53p2a4
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp0d53p2a4/tfx_user_code_Transform-0.0+137bde4ed5143d1edc401ce8ced6b409a8617414fff6073a174b3bad49b88838-py3.9.egg-info
running install_scripts
creating /tmp/tmp0d53p2a4/tfx_user_code_Transform-0.0

/opt/conda/envs/mlops-tfx/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipeline/sentiment_pipeline/_wheels/tfx_user_code_Transform-0.0+137bde4ed5143d1edc401ce8ced6b409a8617414fff6073a174b3bad49b88838-py3-none-any.whl


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing ./pipeline/sentiment_pipeline/_wheels/tfx_user_code_Transform-0.0+137bde4ed5143d1edc401ce8ced6b409a8617414fff6073a174b3bad49b88838-py3-none-any.whl


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [16]:
# Retrieve and print the path to the generated schema.
schema_path = schema_gen.outputs['schema'].get()[0].uri
print(schema_path)

pipeline/sentiment_pipeline/SchemaGen/schema/3


## **5. Model Training and Tuning**
### **Hyperparameter Tuning**
- **Purpose**: Optimize model hyperparameters for better performance.
- **Component**: `Tuner` (uses KerasTuner's Hyperband strategy).
- **Search Space**:
  - **Embedding dimensions** - `min_value=32, max_value=128, step=32`
  - **Spatial dropout rates** - `min_value=0, max_value=0.5, step=0.1`
  - **Dense layer units** - `min_value=32, max_value=128, step=32`
  - **Dropout rates** - `min_value=0, max_value=0.5, step=0.1`
  - **Learning rate** - `1e-2, 1e-3, 1e-4`

### **Model Parameters**
- **Vocabulary Size**: `10,000` 
- **Sequence Lenght**: `100` 

### **Model Design**
- **Architecture**:
  - **Input**: Accepts raw text input.
  - **Reshape**: Converts input shape for processing.
  - **Embedding**: Represents text as dense semantic vectors.
  - **Spatial Dropout**: Regularizes embedding weights.
  - **Global Average Pooling**: Reduces dimensionality of embeddings.
  - **Dense**: Applies ReLU activation for non-linear feature transformation.
  - **Dropout**: Prevents overfitting by randomly zeroing some weights.
  - **Output**: Produces sentiment predictions with softmax activation.
- **Component**: `Trainer` trains the model with optimal hyperparameters.
- **Outputs**: A trained and tuned model.

In [17]:
# Define the file name for the tuner module.
TUNER_MODULE_FILE = 'sentiment_tuner.py'

In [18]:
%%writefile {TUNER_MODULE_FILE}

# Import necessary libraries and modules.
from typing import NamedTuple, Dict, Text, Any, List
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs

# Import Keras and KerasTuner components.
from keras.models import Sequential
from keras.layers import (
    InputLayer,
    Reshape,
    TextVectorization,
    Embedding,
    SpatialDropout1D,
    GlobalAveragePooling1D,
    Dropout,
    Dense,
)
from keras.callbacks import EarlyStopping
import kerastuner as kt
from keras_tuner.engine import base_tuner

# Import constants and utility functions from the transform module.
from sentiment_transform import FEATURE_KEY, LABEL_KEY, transformed_name

# Define constants for vocabulary size and sequence length.
VOCAB_SIZE = 10000
SEQ_LENGTH = 100
EPOCHS = 10

# Initialize a TextVectorization layer.
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQ_LENGTH
)

# Define the result type for the tuner function.
TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner), 
    ('fit_kwargs', Dict[Text, Any])
])

# Configure early stopping to prevent overfitting.
early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=3,
    verbose=1, 
    mode='max'
)

# Function to read compressed TFRecord files.
def gzip_reader_fn(filenames):
    '''Loads compressed data.'''
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Function to prepare input data for training and evaluation.
def input_fn(
        file_pattern,
        tf_transform_output,
        num_epochs,
        batch_size=128
    ) -> tf.data.Dataset:
    '''Get post_transform feature and create batches of data.'''

    # Get transformed feature specifications.
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create batched dataset.
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )

    return dataset

# Function to define and build the model with tunable hyperparameters.
def model_builder(hp):
    '''Builds the model and sets up the hyperparameters to tune.'''
    
    # Define the model architecture.
    model = Sequential([
        InputLayer(
            input_shape=(1,),
            dtype=tf.string, 
            name=transformed_name(FEATURE_KEY)
        ),
        Reshape(()), 
        vectorize_layer,
        Embedding(
            VOCAB_SIZE, 
            hp.Int('embedding_dim', min_value=32, max_value=128, step=32),
            name='embedding'
        ),
        SpatialDropout1D(
            hp.Float('spatial_dropout_rate', min_value=0, max_value=0.5, step=0.1)
        ),
        GlobalAveragePooling1D(),
        Dense(
            hp.Int('dense_units', min_value=32, max_value=128, step=32),
            activation='relu'
        ),
        Dropout(hp.Float('dropout_rate', min_value=0, max_value=0.5, step=0.1)),
        Dense(3, activation='softmax')
    ])
    
    # Compile the model.
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice(
                'learning_rate', 
                values=[1e-2, 1e-3, 1e-4]
            )
        ),
        metrics=['accuracy']
    )

    return model
        
# Function to configure the tuner.
def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    '''Build the tuner using the KerasTuner API.'''
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Prepare training and validation datasets.
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, EPOCHS)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, EPOCHS)

    # Adapt the vectorization layer with training data.
    vectorize_layer.adapt([
        j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]
    ])

    # Initialize the tuner.
    tuner = kt.Hyperband(
        model_builder,
        objective='val_accuracy',
        max_epochs=EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name='sentiment_analysis_kt_hyperband'
    )

    # Return the tuner and fit arguments.
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'callbacks': [early_stopping],
            'x': train_set,
            'validation_data': val_set
        }
    )


Writing sentiment_tuner.py


In [19]:
# Initialize the Tuner component with the specified module file and inputs.
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

# Run the Tuner component to perform hyperparameter tuning.
interactive_context.run(tuner)


Trial 30 Complete [00h 01m 04s]
val_accuracy: 0.8436508178710938

Best val_accuracy So Far: 0.8539682626724243
Total elapsed time: 00h 08m 45s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipeline/sentiment_pipeline/.temp/6/sentiment_analysis_kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
embedding_dim: 96
spatial_dropout_rate: 0.5
dense_units: 96
dropout_rate: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8539682626724243
Trial summary
Hyperparameters:
embedding_dim: 64
spatial_dropout_rate: 0.4
dense_units: 64
dropout_rate: 0.0
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8500000238418579
Trial summary
Hyperparameters:
embedding_dim: 64
spatial_dropout_rate: 0.2
dense_units: 96
dropout_rate: 0.2
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0005
Score: 0.8476190567016602
Trial summary
Hyperparameters:
embedding_dim: 64
spatial_dropout_rate: 0.2
dense_units: 96
dropout_rate: 0.2
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_e

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### **Tuner Results**
The Tuner component completed the hyperparameter optimization process, testing multiple configurations to identify the best-performing model. Below are the results from the tuning process:

- **Best Model Summary**
  - **Validation Accuracy**: `0.8539`
  - **Elapsed Time**: `8 minutes 45 seconds`
  - **Best Hyperparameters**:
    - **Embedding Dimension**: `96`
    - **Spatial Dropout Rate**: `0.5`
    - **Dense Layer Units**: `96`
    - **Dropout Rate**: `0.3`
    - **Learning Rate**: `0.001`
  - **Tuning Strategy**: Hyperband with bracket and round optimization.

In [20]:
# Define the file name for the trainer module.
TRAINER_MODULE_FILE = 'sentiment_trainer.py'

In [21]:
%%writefile {TRAINER_MODULE_FILE}

# Import TensorFlow, TFX, and utilities.
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs
import os

# Import Keras components.
from keras.models import Sequential
from keras.layers import (
    InputLayer,
    Reshape,
    TextVectorization,
    Embedding,
    SpatialDropout1D,
    GlobalAveragePooling1D,
    Dropout,
    Dense,
)
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

# Import constants and utilities from the transform and tuner modules.
from sentiment_transform import FEATURE_KEY, LABEL_KEY, transformed_name
from sentiment_tuner import (
    VOCAB_SIZE, 
    SEQ_LENGTH, 
    EPOCHS,
    vectorize_layer, 
    gzip_reader_fn, 
    input_fn
)

# Define the model architecture with tunable hyperparameters.
def model_builder(hp):
    '''Build the machine learning model.'''
    
    model = Sequential([
        InputLayer(
            input_shape=(1,),
            dtype=tf.string, 
            name=transformed_name(FEATURE_KEY)
        ),
        Reshape(()), 
        vectorize_layer,
        Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding'),
        SpatialDropout1D(hp['spatial_dropout_rate']),
        GlobalAveragePooling1D(),
        Dense(hp['dense_units'], activation='relu'),
        Dropout(hp['dropout_rate']),
        Dense(3, activation='softmax')
    ])

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics=['accuracy']
    )

    model.summary()

    return model

# Define the serving function for the model.
def _get_serve_tf_examples_fn(model, tf_transform_output):
    '''Prepare the model for serving.'''
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_example):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(
            serialized_tf_example, 
            feature_spec
        )
        transformed_features = model.tft_layer(parsed_features)

        # Generate predictions using transformed features.
        return model(transformed_features)

    return serve_tf_examples_fn

# Define the main function to train and save the model.
def run_fn(fn_args: FnArgs) -> None:
    '''Run the model training and evaluation.'''
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    # Configure callbacks for training.
    tensorboard = TensorBoard(log_dir=log_dir, update_freq='batch')
    early_stopping = EarlyStopping(
        monitor='val_accuracy', 
        patience=5,
        verbose=1,
        mode='max', 
        restore_best_weights=True
    )
    model_checkpoint = ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max'
    )

    # Load the transform output.
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Prepare training and validation datasets.
    train_set = input_fn(fn_args.train_files, tf_transform_output, EPOCHS)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, EPOCHS)

    # Adapt the TextVectorization layer using training data.
    vectorize_layer.adapt([
        j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]
    ])

    # Build the model using hyperparameters.
    hp = fn_args.hyperparameters['values']
    model = model_builder(hp)

    # Train the model with specified callbacks.
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard, early_stopping, model_checkpoint],
        epochs=EPOCHS
    )

    # Define and save model signatures for serving.
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir, 
        save_format='tf', 
        signatures=signatures
    )


Writing sentiment_trainer.py


In [22]:
# Initialize the Trainer component with the specified module file and inputs.
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

# Run the Trainer component to train and evaluate the model.
interactive_context.run(trainer)


/opt/conda/envs/mlops-tfx/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying sentiment_trainer.py -> build/lib
copying sentiment_transform.py -> build/lib
copying sentiment_tuner.py -> build/lib
installing to /tmp/tmpa8jn78xt
running install
running install_lib
copying build/lib/sentiment_transform.py -> /tmp/tmpa8jn78xt
copying build/lib/sentiment_tuner.py -> /tmp/tmpa8jn78xt
copying build/lib/sentiment_trainer.py -> /tmp/tmpa8jn78xt
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpa8jn78xt/tfx

INFO:tensorflow:Assets written to: pipeline/sentiment_pipeline/Trainer/model/7/Format-Serving/assets


860/860 [==============================] - 7s 8ms/step - loss: 0.5263 - accuracy: 0.7824 - val_loss: 0.4173 - val_accuracy: 0.8540
Epoch 2/10
858/860 [============================>.] - ETA: 0s - loss: 0.2565 - accuracy: 0.9033
Epoch 2: val_accuracy did not improve from 0.85397
860/860 [==============================] - 6s 6ms/step - loss: 0.2566 - accuracy: 0.9033 - val_loss: 0.4764 - val_accuracy: 0.8429
Epoch 3/10
855/860 [============================>.] - ETA: 0s - loss: 0.1948 - accuracy: 0.9264
Epoch 3: val_accuracy did not improve from 0.85397
860/860 [==============================] - 6s 6ms/step - loss: 0.1947 - accuracy: 0.9264 - val_loss: 0.5569 - val_accuracy: 0.8278
Epoch 4/10
855/860 [============================>.] - ETA: 0s - loss: 0.1638 - accuracy: 0.9389
Epoch 4: val_accuracy did not improve from 0.85397
860/860 [==============================] - 6s 6ms/step - loss: 0.1638 - accuracy: 0.9389 - val_loss: 0.6556 - val_accuracy: 0.8246
Epoch 5/10
856/860 [===============

INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipeline/sentiment_pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline/sentiment_pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [23]:
# Initialize the Model Resolver component to fetch the latest blessed model.
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

# Run the Model Resolver component to retrieve the latest blessed model.
interactive_context.run(model_resolver)


ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## **6. Model Evaluation**
- **Objective**: Validate the trained model and ensure performance reliability.
- **Component**: `Evaluator`.
- **Metrics**:
  - Overall accuracy.
  - Sentiment slice-based analysis.
- **Outputs**:
  - Evaluation results visualized as slicing metrics and fairness indicators.
  - Performance thresholds to determine if the model is production-ready.

In [24]:
# Configure evaluation settings for the model.
eval_config = tfma.EvalConfig(
    
    # Define model specifications with the target label key.
    model_specs=[
        tfma.ModelSpec(label_key='sentiment')
    ],
    
    # Specify slicing configurations for both overall and by feature 'sentiment.'
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['sentiment'])
    ],
    
    # Define evaluation metrics and thresholds.
    metrics_specs=[
        tfma.MetricsSpec(
            per_slice_thresholds={
                'accuracy': tfma.PerSliceMetricThresholds(
                    thresholds=[
                        tfma.PerSliceMetricThreshold(
                            slicing_specs=[tfma.SlicingSpec()],
                            threshold=tfma.MetricThreshold(
                                value_threshold=tfma.GenericValueThreshold(
                                    lower_bound={'value': 0.5}
                                ),
                                change_threshold=tfma.GenericChangeThreshold(
                                    direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                    absolute={'value': -1e-10}
                                )
                            )
                        )
                    ]
                )
            }
        )
    ]
)


In [25]:
# Initialize the Evaluator component for model evaluation.
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

# Run the Evaluator component to assess the model's performance.
interactive_context.run(evaluator)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [26]:
# Load and display evaluation results.
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)

# Render slicing metrics to visualize performance across data slices.
tfma.view.render_slicing_metrics(tfma_result)

# Render the fairness indicator widget to evaluate fairness across slices.
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)


FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

## **7. Model Deployment**
- **Component**: `Pusher`.
- **Process**:
  - Validates the model based on evaluation metrics.
  - Deploys the model to a serving directory for real-time prediction.
- **Output**: Production-ready model stored in `serving_model_dir`.

In [27]:
# Initialize the Pusher component to deploy the trained and validated model.
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='serving_model_dir/sentiment-analysis-model'
        )
    )
)

# Execute the Pusher component to push the model to the serving directory.
interactive_context.run(pusher)


ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## **8. Project Summary**
The end-to-end pipeline ensures robust sentiment analysis for Indonesian text by leveraging TFX components. It incorporates data validation, preprocessing, hyperparameter tuning, and model evaluation to produce a reliable and scalable solution ready for deployment.

For deploying the trained model locally using Docker and performing single and batch predictions, refer to the instructions provided in **testing.ipynb**. The document contains steps for setting up a Docker container, deploying the model, and testing predictions.